password: 4n=KTc8mxoKwp=UCKfw_
token: eyJ2ZXIiOiI4LjYuMiIsImFkciI6WyIxOTIuMTY4LjAuMTExOjkyMDAiXSwiZmdyIjoiYzE0OWY2ZDllOWU4MjQ3ZjE0MzdmMDRlMjE3MDBkNmQzNWNmYThjYTk2ZmM5OTY1YTBjYjZlMjNmNjgxYTJhNCIsImtleSI6InY4Q3Fqb2dCbDBFRzZNRndBQS1qOlpTUFczaGpKUU0tdlVyWndYZFNFU2cifQ==

In [44]:
%pip install elasticsearch

Note: you may need to restart the kernel to use updated packages.


## Kết nối Elastic

In [45]:
from elasticsearch import Elasticsearch
import os

HOST = "https://localhost:9200"
ELASTIC_USER = "elastic"
# The password for the 'elastic' user generated by Elasticsearch
ELASTIC_PASSWORD = "4n=KTc8mxoKwp=UCKfw_"
# The path of ca certificates 
CA_CERTS = os.path.realpath(os.path.join(os.path.dirname(__name__), 'elasticsearch-8.6.2', 'config', 'certs', 'http_ca.crt'))

# Create the client instance
es = Elasticsearch(
    hosts = HOST,
    ca_certs = CA_CERTS,
    http_auth = (ELASTIC_USER, ELASTIC_PASSWORD),
)
index_name = "product"
type_name = "_doc"
path = "documents/"
es.info()

C:\Users\totha\AppData\Local\Temp\ipykernel_14112\1753550304.py:11: DeprecationWarning: The 'http_auth' parameter is deprecated. Use 'basic_auth' or 'bearer_auth' parameters instead
  es = Elasticsearch(


ObjectApiResponse({'name': 'FONGTO', 'cluster_name': 'elasticsearch', 'cluster_uuid': 'GggxXucnT8abgm2hfSsrKQ', 'version': {'number': '8.6.2', 'build_flavor': 'default', 'build_type': 'zip', 'build_hash': '2d58d0f136141f03239816a4e360a8d17b6d8f29', 'build_date': '2023-02-13T09:35:20.314882762Z', 'build_snapshot': False, 'lucene_version': '9.4.2', 'minimum_wire_compatibility_version': '7.17.0', 'minimum_index_compatibility_version': '7.0.0'}, 'tagline': 'You Know, for Search'})

### Tạo index

In [46]:
import json

if es.indices.exists(index=index_name):
    es.indices.delete(index=index_name)
else:
    print("index does not exists")
es.indices.create(index=index_name)

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'product'})

### Định nghĩa dữ liệu

In [47]:
body = {
    "properties": {
        "asin": {"type": "keyword"},
        "brand": {
            "type": "text",
            "term_vector": "with_positions_offsets"
        },
        "stars": {"type": "text"},
        "reviewsCount": {"type": "text"},
        "thumbnailImage": {"type": "text"},
        "description": {"type": "text"},
        "url": {"type":"text"},
        "price": {
            "properties":{
                "value": {"type": "text"},
                "currency": {"type": "text"},
            }
        },
        "title": {
            "type": "text",
            "term_vector": "with_positions_offsets"
        },
        "breadCrumbs": {"type":"text","term_vector": "with_positions_offsets"},
    }
}    
es.indices.put_mapping(index = index_name, body=body)

C:\Users\totha\AppData\Local\Temp\ipykernel_14112\3304693643.py:26: DeprecationWarning: The 'body' parameter is deprecated and will be removed in a future version. Instead use individual parameters.
  es.indices.put_mapping(index = index_name, body=body)


ObjectApiResponse({'acknowledged': True})

### Load dữ liệu vào index

In [48]:
import os
i = 0

for filename in os.listdir(path):
    if filename.endswith('.json'):
        with open(path + filename) as open_file:
            content = open_file.read()
            data = json.loads(content)
            try:
                data['price']['value'] = float(data['price']['value'])
            except Exception as ex:
                data['price']['value'] = 5.9
            es.index(index=index_name, ignore=400, id=i, body=data)
            i = i + 1

C:\Users\totha\AppData\Local\Temp\ipykernel_14112\555523147.py:13: DeprecationWarning: Passing transport options in the API method is deprecated. Use 'Elasticsearch.options()' instead.
  es.index(index=index_name, ignore=400, id=i, body=data)
C:\Users\totha\AppData\Local\Temp\ipykernel_14112\555523147.py:13: DeprecationWarning: The 'body' parameter is deprecated and will be removed in a future version. Instead use the 'document' parameter. See https://github.com/elastic/elasticsearch-py/issues/1698 for more information
  es.index(index=index_name, ignore=400, id=i, body=data)


### Kiểm tra

In [49]:
es.get(index=index_name, id=1)["_source"]

{'title': 'Fiskars SoftGrip Scissors - Contoured Performance All Purpose - 8" Stainless Steel - Straight Paper Scissors for Office, and Arts and Crafts - Grey',
 'asin': 'B002YIP97K',
 'brand': 'Gray',
 'stars': '4.8',
 'reviewsCount': '14,111 ratings',
 'thumbnailImage': 'https://m.media-amazon.com/images/I/51LOf8l48VL._AC_SY355_.jpg',
 'breadCrumbs': ['Arts, Crafts & Sewing',
  '›',
  'Crafting',
  '›',
  'Craft Supplies',
  '›',
  'Cutting Tools',
  '›',
  'Scissors'],
 'description': 'Sharp and comfortable all-purpose scissors',
 'price': {'value': 6.48, 'currency': '$'},
 'url': 'https://www.amazon.com/Fiskars-01-004761J-Softgrip-Scissors-Stainless/dp/B002YIP97K/ref=sr_1_10?qid=1684768565&s=arts-crafts-intl-ship&sr=1-10'}

In [50]:
mapping = es.indices.get_mapping(index=index_name)
print(mapping)

{'product': {'mappings': {'properties': {'asin': {'type': 'keyword'}, 'brand': {'type': 'text', 'term_vector': 'with_positions_offsets'}, 'breadCrumbs': {'type': 'text', 'term_vector': 'with_positions_offsets'}, 'description': {'type': 'text'}, 'price': {'properties': {'currency': {'type': 'text'}, 'value': {'type': 'text'}}}, 'reviewsCount': {'type': 'text'}, 'stars': {'type': 'text'}, 'thumbnailImage': {'type': 'text'}, 'title': {'type': 'text', 'term_vector': 'with_positions_offsets'}, 'url': {'type': 'text'}}}}}


## Cài đặt phân tích

In [51]:
body = {
    "settings": {
        "analysis": {
            "analyzer": {
                "my_analyzer": {
                    "tokenizer": "standard",
                    "filter": ["lowercase"]
                }
            },
            "tokenizer": {
                "my_tokenizer": {
                    "type": "pattern",
                    "pattern": "\"((?:\\\\\"|[^\"]|\\\\\")+)\" ",
                    "group": 1
                }
            }
    }
  }
}
es.indices.close(index=index_name)
es.indices.put_settings(index=index_name, body=body)
es.indices.open(index=index_name)

C:\Users\totha\AppData\Local\Temp\ipykernel_14112\1436664397.py:21: DeprecationWarning: The 'body' parameter is deprecated and will be removed in a future version. Instead use the 'settings' parameter. See https://github.com/elastic/elasticsearch-py/issues/1698 for more information
  es.indices.put_settings(index=index_name, body=body)


ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True})

## Tạo hàm Ranking

In [52]:
body = {
    "settings": {
        "index": {
            "similarity": {
                "my_similarity": {
                    "type": "DFR",
                    "basic_model": "g",
                    "after_effect": "l",
                    "normalization": "h2",
                    "normalization.h2.c": "3.0"
                }
            }
        }
    }
}
es.indices.close(index=index_name)
es.indices.put_settings(index=index_name, body=body)
es.indices.open(index=index_name)

C:\Users\totha\AppData\Local\Temp\ipykernel_14112\2237537309.py:17: DeprecationWarning: The 'body' parameter is deprecated and will be removed in a future version. Instead use the 'settings' parameter. See https://github.com/elastic/elasticsearch-py/issues/1698 for more information
  es.indices.put_settings(index=index_name, body=body)


ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True})

## Kiểm tra searching

In [53]:
body = {
    "query": {
        "multi_match": {
            "query": "Glue", 
            "fields": ["brand", "title"], 
            "operator": "and"
        }
    }
}
products = [p['_source'] for p in es.search(index=index_name, body=body)["hits"]["hits"]]
print(products)

[{'title': 'Colorations Washable Clear Glue, 1 Gallon, Dries Clear, Gluing, Crafts, School Glue, Home Glue, Office Glue, Craft Projects, Washable Glue, Non Toxic Glue, Homeschool, Home School Use', 'asin': '4.5\n881 ratings\n4.5 out of 5 stars', 'brand': 'Colorations', 'stars': '4.5', 'reviewsCount': '881 ratings', 'thumbnailImage': 'https://m.media-amazon.com/images/I/71fLSZVLcjL._AC_SX450_.jpg', 'breadCrumbs': ['Arts, Crafts & Sewing', '›', 'Crafting', '›', 'Craft Supplies', '›', 'Adhesives', '›', 'Art Glues & Pastes'], 'description': 'Colorations (R) Washable Clear Glue that dries clear, nontoxic, and very washable! Great for making slime, art projects and collaging! Value size and acid free this glue is ideal for school, home and office. Great for Homeschool, or home school use for art projects and experiments.', 'price': {'value': 20.99, 'currency': '$'}, 'url': 'https://www.amazon.com/Colorations-NGL-Washable-Clear-Glue/dp/B01HZMNDFO/ref=sr_1_1063?qid=1684769033&s=arts-crafts-int

C:\Users\totha\AppData\Local\Temp\ipykernel_14112\713086281.py:10: DeprecationWarning: The 'body' parameter is deprecated and will be removed in a future version. Instead use individual parameters.
  products = [p['_source'] for p in es.search(index=index_name, body=body)["hits"]["hits"]]
